In [4]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from lib.utils import *

def get_basic_cm_stat(idx, stat):
        # Inupt: stat in shape (1, attribute, 8)
        mtp, mfp, mfn, mtn = stat[0,idx,0:4]
        ftp, ffp, ffn, ftn = stat[0,idx,4:8]
        # accuracy
        macc = (mtp+mtn)/(mtp+mfp+mfn+mtn)
        facc = (ftp+ftn)/(ftp+ffp+ffn+ftn)
        tacc = (mtp+mtn+ftp+ftn)/(mtp+mfp+mfn+mtn+ftp+ffp+ffn+ftn)
        # fairness
        mtpr, mtnr = mtp/(mtp+mfn), mtn/(mtn+mfp)
        ftpr, ftnr = ftp/(ftp+ffn), ftn/(ftn+ffp)
        equality_of_opportunity = abs(mtpr-ftpr)
        equalized_odds = abs(mtpr-ftpr) + abs(mtnr-ftnr)
        return macc, facc, tacc, equality_of_opportunity, equalized_odds

def get_basic_cm_stat_list(idx, stat):
    # Input: stat in shape (epoch, attribute, 8)
    macc_list, facc_list, tacc_list, eoo_list, eo_list = list(), list(), list(), list(), list()
    for e in range(stat.shape[0]):
        macc, facc, tacc, equality_of_opportunity, equalized_odds = get_basic_cm_stat(idx, stat[e:e+1,:,:])
        macc_list.append(macc)
        facc_list.append(facc)
        tacc_list.append(tacc)
        eoo_list.append(equality_of_opportunity)
        eo_list.append(equalized_odds)
    return macc_list, facc_list, tacc_list, eoo_list, eo_list

def show_celeba_val_stat(val_stat, name='default', root_folder='./eval/celeba'):
    folder = Path(root_folder)
    folder.mkdir(parents=True, exist_ok=True)
    path = folder / f"{name}.png"
    fig, axs  = plt.subplots(2,4, figsize=(18,8))
    attributes = ['Attractive', 'High_Cheekbones', 'Mouth_Slightly_Open', 'Smiling']
    xs = np.linspace(0, val_stat.shape[0]-1, val_stat.shape[0])
    for attr in range(val_stat.shape[1]): # attribute
        macc_list, facc_list, tacc_list, eoo_list, eo_list = get_basic_cm_stat_list(attr, val_stat)
        axs[0][attr].set_title(attributes[attr])
        macc, = axs[0][attr].plot(xs, macc_list)
        facc, = axs[0][attr].plot(xs, facc_list)
        tacc, = axs[0][attr].plot(xs, tacc_list)
        axs[0][attr].set_xlabel('Epochs')
        axs[0][attr].set_ylabel('Accuracy')
        axs[0][attr].legend((macc, facc, tacc), ('Male', 'Female', 'Total'), loc='lower right')
        axs[0][attr].set_ylim([0.0, 1.0])
        #
        eoo, = axs[1][attr].plot(xs, eoo_list)
        eo, = axs[1][attr].plot(xs, eo_list)
        axs[1][attr].set_xlabel('Epochs')
        axs[1][attr].set_ylabel('Fairness, (lower the better)')
        axs[1][attr].legend((eoo, eo), ('equality of opportunity', 'equalized odds'), loc='upper right')
        axs[1][attr].set_ylim([0.0, 1.0])

    fig.tight_layout()
    fig.savefig(path,)
    plt.close(fig)

# load the validation statistics
# the file name might start with: {seed}_{dataset name}_val
# val_stat = load_stats(f'33907_CelebA_noise_pq_val', root_folder='/tmp2/aislab/makila/noise')
# show_celeba_val_stat(val_stat, name='CelebA_noise_pq')

def show_agemodel_val_stat(val_stat, name='default', root_folder='./eval/agemodel'):
    folder = Path(root_folder)
    folder.mkdir(parents=True, exist_ok=True)
    path = folder / f"{name}.png"
    fig, axs  = plt.subplots(2,1, figsize=(5,8))
    xs = np.linspace(0, val_stat.shape[0]-1, val_stat.shape[0])
    # only have 1 attribute (age)
    macc_list, facc_list, tacc_list, eoo_list, eo_list = get_basic_cm_stat_list(0, val_stat)
    axs[0].set_title('Is age greater than 30?')
    macc, = axs[0].plot(xs, macc_list)
    facc, = axs[0].plot(xs, facc_list)
    tacc, = axs[0].plot(xs, tacc_list)
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Accuracy')
    axs[0].legend((macc, facc, tacc), ('Male', 'Female', 'Total'), loc='lower right')
    axs[0].set_ylim([0.5, 1.0])
    #
    eoo, = axs[1].plot(xs, eoo_list)
    eo, = axs[1].plot(xs, eo_list)
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Fairness, (lower the better)')
    axs[1].legend((eoo, eo), ('equality of opportunity', 'equalized odds'), loc='upper right')
    axs[1].set_ylim([0.0, 1.0])

    fig.tight_layout()
    fig.savefig(path,)
    plt.close(fig)

def show_agemodel_val_status(val_stat, name='default', root_folder='./eval/agemodel'):
    folder = Path(root_folder)
    folder.mkdir(parents=True, exist_ok=True)
    path = folder / f"{name}.png"
    # only have 1 attribute (age)
    macc_list, facc_list, tacc_list, eoo_list, eo_list = get_basic_cm_stat_list(0, val_stat)
    fig, axs  = plt.subplots(2,1, figsize=(5,8))
    axs[0].set_title('Equality of Opportunity')
    eoo, = axs[0].plot(list(map(lambda x: 1.0-x, eoo_list)), tacc_list)
    axs[0].set_xlabel('Fairness')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_box_aspect(1)
    axs[0].set_xlim([0.5, 1.0])
    axs[0].set_ylim([0.5, 1.0])
    #
    axs[1].set_title('Equalized Odds')
    eo = axs[1].plot(list(map(lambda x: 1.0-x, eo_list)), tacc_list)
    axs[1].set_xlabel('Fairness')
    axs[1].set_ylabel('Accuracy')
    axs[1].set_box_aspect(1)
    axs[1].set_xlim([0.5, 1.0])
    axs[1].set_ylim([0.5, 1.0])
    #
    fig.tight_layout()
    fig.savefig(path,)
    plt.close(fig)

def animate_agemodel_val_status(val_stat, name='default', root_folder='./eval/agemodel'):
    folder = Path(root_folder)
    folder.mkdir(parents=True, exist_ok=True)
    path = folder / f"{name}.gif"
    # only have 1 attribute (age)
    macc_list, facc_list, tacc_list, eoo_list, eo_list = get_basic_cm_stat_list(0, val_stat)
    fig, axs = plt.subplots(2,1, figsize=(5,8))
    axs[0].set_title('Equality of Opportunity')
    axs[0].set_xlabel('Fairness')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_box_aspect(1)
    axs[0].set_xlim([0.5, 1.0])
    axs[0].set_ylim([0.5, 1.0])
    eoo, = axs[0].plot([], [], zorder=0)
    dot_eoo = axs[0].scatter([], [], c='blue', zorder=2)
    #
    axs[1].set_title('Equalized Odds')
    axs[1].set_xlabel('Fairness')
    axs[1].set_ylabel('Accuracy')
    axs[1].set_box_aspect(1)
    axs[1].set_xlim([0.5, 1.0])
    axs[1].set_ylim([0.5, 1.0])
    eo, = axs[1].plot([], [], zorder=0)
    dot_eo = axs[1].scatter([], [], c='blue', zorder=2)

    def animate(i):
        eoo.set_data(list(map(lambda x: 1.0-x, eoo_list[0:i+1])), tacc_list[0:i+1])
        dot_eoo.set_offsets((1.0-eoo_list[i], tacc_list[i]))
        eo.set_data(list(map(lambda x: 1.0-x, eo_list[0:i+1])), tacc_list[0:i+1])
        dot_eo.set_offsets((1.0-eo_list[i], tacc_list[i]))
    
    fig.tight_layout()
    animated = animation.FuncAnimation(fig, animate, frames=val_stat.shape[0], interval=200, repeat_delay=1000)
    animated.save(path, writer='pillow')
    plt.close(fig)

def agemodel_val_status(eop_val_stats, eo_val_stats, name='default', root_folder='./eval/agemodel'):
    folder = Path(root_folder)
    folder.mkdir(parents=True, exist_ok=True)
    path = folder / f"{name}.png"
    # show equality of opportunity and equalized odds for their separated, dedicated experiment
    fig, axs = plt.subplots(2,1, figsize=(5,8))
    # equality of opportunity
    macc_list, facc_list, tacc_list, eoo_list, _ = get_basic_cm_stat_list(0, eop_val_stats)
    axs[0].set_title('Equality of Opportunity')
    eoo, = axs[0].plot(list(map(lambda x: 1.0-x, eoo_list)), tacc_list)
    axs[0].set_xlabel('Fairness')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_box_aspect(1)
    axs[0].set_xlim([0.5, 1.0])
    axs[0].set_ylim([0.5, 1.0])
    # equalized odds
    macc_list, facc_list, tacc_list, _, eo_list = get_basic_cm_stat_list(0, eo_val_stats)
    axs[1].set_title('Equalized Odds')
    eo = axs[1].plot(list(map(lambda x: 1.0-x, eo_list)), tacc_list)
    axs[1].set_xlabel('Fairness')
    axs[1].set_ylabel('Accuracy')
    axs[1].set_box_aspect(1)
    axs[1].set_xlim([0.5, 1.0])
    axs[1].set_ylim([0.5, 1.0])
    #
    fig.tight_layout()
    fig.savefig(path,)
    plt.close(fig)
    
def agemodel_val_animated(eop_val_stats, eo_val_stats, name='default', root_folder='./eval/agemodel'):
    folder = Path(root_folder)
    folder.mkdir(parents=True, exist_ok=True)
    path = folder / f"{name}.png"
    # show equality of opportunity and equalized odds for their separated, dedicated experiment
    fig, axs = plt.subplots(2,1, figsize=(5,8))
    # equality of opportunity
    macc_list, facc_list, eoo_tacc_list, eoo_list, _ = get_basic_cm_stat_list(0, eop_val_stats)
    fig, axs = plt.subplots(2,1, figsize=(5,8))
    axs[0].set_title('Equality of Opportunity')
    axs[0].set_xlabel('Fairness')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_box_aspect(1)
    axs[0].set_xlim([0.5, 1.0])
    axs[0].set_ylim([0.5, 1.0])
    eoo, = axs[0].plot([], [], zorder=0)
    dot_eoo = axs[0].scatter([], [], c='blue', zorder=2)
    # equalized odds
    macc_list, facc_list, eo_tacc_list, _, eo_list = get_basic_cm_stat_list(0, eo_val_stats)
    axs[1].set_title('Equalized Odds')
    axs[1].set_xlabel('Fairness')
    axs[1].set_ylabel('Accuracy')
    axs[1].set_box_aspect(1)
    axs[1].set_xlim([0.5, 1.0])
    axs[1].set_ylim([0.5, 1.0])
    eo, = axs[1].plot([], [], zorder=0)
    dot_eo = axs[1].scatter([], [], c='blue', zorder=2)

    def animate(i):
        eoo.set_data(list(map(lambda x: 1.0-x, eoo_list[0:i+1])), eoo_tacc_list[0:i+1])
        dot_eoo.set_offsets((1.0-eoo_list[i], eoo_tacc_list[i]))
        eo.set_data(list(map(lambda x: 1.0-x, eo_list[0:i+1])), eo_tacc_list[0:i+1])
        dot_eo.set_offsets((1.0-eo_list[i], eo_tacc_list[i]))
    
    fig.tight_layout()
    animated = animation.FuncAnimation(fig, animate, frames=eop_val_stats.shape[0], interval=200, repeat_delay=1000)
    animated.save(path, writer='pillow')
    plt.close(fig)

In [2]:
# UTKFace Age (dedicated)
noise_stat_root = Path('/tmp2/aislab/makila/noise_stats')
# these 3 list should have the same length
# eop_stat_name_list = ['UTKFaceAgeDirect_fix_EOP', 'UTKFaceAgeDirect_Dynamic_EOP', 'UTKFaceAgeBCEmasking_EOP', 'UTKFaceAgePOptim_0_EOP', 'UTKFaceAgePOptim_1_EOP']
# eo_stat_name_list = ['UTKFaceAgeDirect_fix_EO', 'UTKFaceAgeDirect_Dynamic_EO', 'UTKFaceAgeBCEmasking_EO', 'UTKFaceAgePOptim_0_EO', 'UTKFaceAgePOptim_1_EO']
# stat_name_list = ['UTKFaceAgeDirect_fix', 'UTKFaceAgeDirect_Dynamic', 'UTKFaceAgeBCEmasking', 'UTKFaceAgePOptim_0', 'UTKFaceAgePOptim_1']
eop_stat_name_list = ['UTKFaceAgeEOOPOptim_0', 'UTKFaceAgeEOOPOptim_1']
eo_stat_name_list = ['UTKFaceAgeEOPOptim_0', 'UTKFaceAgeEOPOptim_1']
stat_name_list = ['UTKFaceAgePOptim_0', 'UTKFaceAgePOptim_1']
for idx in range(len(eop_stat_name_list)):
    eop_val_name = eop_stat_name_list[idx]
    eop_val_stat = load_stats(eop_val_name+'_val', root_folder=noise_stat_root/eop_val_name)
    eo_val_name = eo_stat_name_list[idx]
    eo_val_stat = load_stats(eo_val_name+'_val', root_folder=noise_stat_root/eo_val_name)
    agemodel_val_status(eop_val_stat, eo_val_stat, name=f'{stat_name_list[idx]}', root_folder='./eval/utkfaceage')
    # can't draw it if they are in different shape
    # agemodel_val_animated(eop_val_stat, eo_val_stat, name=f'{stat_name_list[idx]}', root_folder='./eval/utkfaceage')

In [5]:
# FairFace Age (dedicated)
noise_stat_root = Path('/tmp2/aislab/makila/noise_stats')
# these 3 list should have the same length
eop_stat_name_list = ['FairFaceAgeDirect_fix_EOP', 'FairFaceAgeDirect_Dynamic_EOP', 'FairFaceAgeBCEmasking_EOP', 'FairFaceAgePOptim_0_EOP', 'FairFaceAgePOptim_1_EOP']
eo_stat_name_list = ['FairFaceAgeDirect_fix_EO', 'FairFaceAgeDirect_Dynamic_EO', 'FairFaceAgeBCEmasking_EO', 'FairFaceAgePOptim_0_EO', 'FairFaceAgePOptim_1_EO']
stat_name_list = ['FairFaceAgeDirect_fix', 'FairFaceAgeDirect_Dynamic', 'FairFaceAgeBCEmasking', 'FairFaceAgePOptim_0', 'FairFaceAgePOptim_1']
for idx in range(len(eop_stat_name_list)):
    eop_val_name = eop_stat_name_list[idx]
    eop_val_stat = load_stats(eop_val_name+'_val', root_folder=noise_stat_root/eop_val_name)
    eo_val_name = eo_stat_name_list[idx]
    eo_val_stat = load_stats(eo_val_name+'_val', root_folder=noise_stat_root/eo_val_name)
    agemodel_val_status(eop_val_stat, eo_val_stat, name=f'{stat_name_list[idx]}', root_folder='./eval/fairfaceage')
    # can't draw it if they are in different shape
    # agemodel_val_animated(eop_val_stat, eo_val_stat, name=f'{stat_name_list[idx]}', root_folder='./eval/fairfaceage')

In [2]:
# UTKFace Age
noise_stat_root = Path('/tmp2/aislab/makila/noise_stats')
stat_name_list = ['UTKFaceAgeDirect', 'UTKFaceAgeDirectDy', 'UTKFaceAgeBCEMasking', 'UTKFaceAgePMAP','UTKFaceAgePMAP_A', 'UTKFaceAgePMAP_A2']
for stat_name in stat_name_list:
    val_stat = load_stats(stat_name+'_val', root_folder=noise_stat_root/stat_name)
    show_agemodel_val_status(val_stat, name=f'{stat_name}', root_folder='./eval/utkface')
    animate_agemodel_val_status(val_stat, name=f'{stat_name}', root_folder='./eval/utkface')

In [ ]:
# stat_name_list = ['CelebA_noise_batch_2_l_1', 'CelebA_noise_batch_2_l_100', 'CelebA_noise_batch_2_e_1', 'CelebA_noise_batch_2_e_100']
stat_name_list = ['CelebA_noise_epoch_2_l_1', 'CelebA_noise_epoch_2_l_100', 'CelebA_noise_epoch_5_l_1', 'CelebA_noise_epoch_5_l_100',
                  'CelebA_noise_epoch_10_l_1', 'CelebA_noise_epoch_10_l_100']

for stat_name in stat_name_list:
    val_stat = load_stats(f'33907_{stat_name}_val', root_folder='/tmp2/aislab/makila/noise')
    show_celeba_val_stat(val_stat, name=f'{stat_name}')

In [ ]:
def get_basic_cm(idx, stat):
        # Inupt: stat in shape (1, attribute, 8)
        mtp, mfp, mfn, mtn = stat[0,idx,0:4]
        ftp, ffp, ffn, ftn = stat[0,idx,4:8]
        return mtp, mfp, mfn, mtn, ftp, ffp, ffn, ftn

def _celeba_val_stat(val_stat, name='default', root_folder='./eval/celeba'):
    folder = Path(root_folder)
    folder.mkdir(parents=True, exist_ok=True)
    path = folder / f"{name}.png"
#     fig, axs  = plt.subplots(2,4, figsize=(18,8))
    attributes = ['Attractive', 'High_Cheekbones', 'Mouth_Slightly_Open', 'Smiling']
    xs = np.linspace(0, val_stat.shape[0]-1, val_stat.shape[0])
    for attr in range(val_stat.shape[1]): # attribute
        print(get_basic_cm(attr, val_stat[110:111]))

val_stat = load_stats(f'33907_CelebA_noise_cm_tp_bo_F_val', root_folder='/tmp2/aislab/makila/noise')
_celeba_val_stat(val_stat)